<a href="https://colab.research.google.com/github/APWright/CSC477-Fall2025/blob/main/Assignment-2/Assignment_2_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2: Data wrangling and Exploratory Visualization



  - To complete this excercise start by saving a copy of this template to google drive using the "Copy to Drive" button

  - In your copy, complete the notebook directly in place at the sections labeled:
  ```python
  # Your code here
  ```

  - Once you have finished the exercise, click the Share button at the top of the notebook. Provide anyone with the link view access, then copy the link and submit it in Canvas.
  
  - Additionally, under File>Download save the notebook file and submit it in Canvas as well.

In this lab we will be using the dataframe library *polars* and the data visualization library *vega-altiar*. If you are already familiar with libraries such as pandas a lot will be transferable, however altair does work meaningfully differently than other common plotting libraries like matplotlib. First we should start by importing the required libraries and downloading the required datasets by running the following cell.

In [ ]:
# @title Import libraries and setup environment
%%capture
%pip install vegafusion vl-convert-python

!git clone https://github.com/APWright/CSC477-Fall2025.git
%cd  CSC477-Fall2025/Assignment-2/


import polars as pl
import altair as alt
import vega_datasets

zipcodes = vega_datasets.data.zipcodes().to_dict('records')
population = pl.read_csv('population.csv')
population_wide = pl.read_csv('population_wide.csv')
pop_untidy = pl.read_csv('pop.csv',infer_schema=False)
gdp_untidy = pl.read_csv('gdp_pcap.csv',infer_schema=False)
emissions_untidy = pl.read_csv('co2_pcap_cons.csv',infer_schema=False)

alt.data_transformers.enable("vegafusion")

# Part 1: Data Wranging withoutout Dataframes

For this exercise, we'll use a dataset of US zipcodes.
The dataset is available in the zipcodes array, which is an array of objects. As an example, here's what the first object looks like:

In [ ]:
zipcodes[0]



For this part we are working through how data wranging and analysis might work without the use of dedicated DataFrame apis. Therefore for the following cells only use built-in python language features such as Using [`map`](https://docs.python.org/3/library/functions.html#map), [`filter`](https://docs.python.org/3/library/functions.html#filter), and [list comprehensions](https://www.w3schools.com/python/python_lists_comprehension.asp).

### 1.1: Find the zipcodes associated with the city of San Luis Obispo.

Cell should output a list of zipcode records.






In [ ]:
answer_1_1 = ...

# Your code here

answer_1_1

### 1.2: How many zip codes are associated with the state of California?

---



Cell should output a number.






In [ ]:
answer_1_2 =  ...

# Your code here

answer_1_2

### 1.3: Derived Fields
Often in visualization, we want to display derived fields, rather than only the fields in the original dataset.

We would like to compute, for each record, the first character of each zipcode. For example, for the zipcode object associated with 93405, we want the following record:


```python
{
  'zip_code': '93401',
  'latitude': 35.265573,
  'longitude': -120.62122,
  'city': 'San Luis Obispo',
  'state': 'CA',
  'county': 'San Luis Obispo'
  'first_digit' : 9 # <--- New field added!
}
  ```

Produce a new list of records that includes a new field called `first_digit` that contains this value.

Expected answer is a list of zipcode objects, with each object now also having a `first_digit` key.

In [ ]:
answer_1_3 = ...

# Your code here

answer_1_3 = list(map(add_first_digit, zipcodes))

### 1.4: Find the maximum zipcode in the US.

[Is this a meaningful value?](https://en.wikipedia.org/wiki/ZIP_Code#Geographic_hierarchy)

In [ ]:
answer_1_4 = ...

# Your code here

answer_1_4

# Part 2: Data Wrangling with Polars

We can do quite a bit with built-in data strucutres like lists, arrays, dicts, and built-in functions like map, filter, and list comprehensions. However, other common data wrangling operations are unavailable through arrays (like group by operations), or would be inefficient when working with large datasets.

That's where [Polars](https://docs.pola.rs/) comes in. Polars is an open source DataFrame library much like (and often interoperable with) [Pandas](https://pandas.pydata.org/) but written in Rust for better performance.

***STOP BEFORE YOU CONTINUE***

Peruse the [polars user guide ](https://docs.pola.rs/user-guide/getting-started/) before moving on with this section, and keep it as well as the [API Reference](https://docs.pola.rs/api/python/stable/reference/index.html) handy. This lab assignment is intentionally light on direct instruction, partly because the linked resources do it better, and partly because in an upper division course you should be used to reading documentation.



Now we can get started and import our zipcode data into a dataframe. The following exercises will ask you to perform some data tasks on the dataframe `df`.


In [ ]:
import polars as pl
df = pl.DataFrame(zipcodes)
df


---



### 2.1: How many zip codes are there in the US? (Or, how many rows are in your data table?)

In [ ]:
answer_2_1 = ...

# Your Code Here

answer_2_1

### 2.2 How many counties does each state have?

 Hint: you will want to look at df.groupby (to group by state), agg (to summarize per state), and n_unique(to find the number of unique county names).

  You'll know your answer is right if you see that California has 58 counties.

In [ ]:
answer_2_2 = ...

# Your Code Here

answer_2_2

### 2.3 Derive a column

Same as you did with the array in Part 1, we'll now derive a column for our data table. Add a new last_digit column to the data table that is simply the last digit of the zip codes, and cast it as a 8 bit integer (`pl.Int8`). Instead of simply viewing the result, save it in a new variable.

It will be useful to have read the documentation on [Expressions](https://docs.pola.rs/user-guide/concepts/expressions-and-contexts/), [Strings](https://docs.pola.rs/user-guide/expressions/strings/), and [Casting](https://docs.pola.rs/user-guide/expressions/casting/).


In [ ]:
answer_2_3 = ...

# Your Code Here

answer_2_3

# A sneak peak of what's to come

So how do we use this data in visualization? For most of this course, we're going to use a visualization library called [Vega-Altair](https://altair-viz.github.io/index.html) that allows us to create powerful interactive graphics using a relatively simple declarative syntax. Polars provides built in support for Altair.

Run the below cell to see a plot of US zip codes. Expand the source code for the cell below and see the comments, along with some nudges to experiment.

In [ ]:
# @title US Zip Codes

import altair as alt
import vega_datasets

# Read in polygons from topojson
states = alt.topo_feature(vega_datasets.data.us_10m.url, feature='states')

#US states background
background = alt.Chart(states).mark_geoshape(
    fill='lightgray',
    stroke='white'
).properties(
    width=500,
    height=300
).project('albersUsa')
"""
The whole chart is projected (roughly, "scaled") using the Albers USA projection.
It takes certain liberties with the locations of Alaska and Hawaii to present the
map in a confined space.
Comment this out to see what happens without it.
Also try different projections, like "albers", "equalEarth", or "mercator".
"""

# plot zip positions on background
points = alt.Chart(df).mark_circle(
    size=10,
    opacity=0.6,
).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    tooltip='county',
    color='first_digit'
)

#Layer points and background
chart = alt.layer(
    background, points
).properties(
    width=500,
    height=500,
    padding=10  # Padding ensures labels fit
).interactive()

chart

# Part 3: Into the Tidyverse

In class we talked about Tidy data. A tidy dataset has three properties:

1.  Each variable has its own column.
1.    Each observation has its own row.
1.    Each value has its own cell.


Consider this table of US population counts from 1850 to 2000. It is tidy, since each observation is a row, and each variable is a column.

The **age** column depicts an **age** *range*, i.e., when age is 0, it means people aged 0–4. When **age** is 5, it means people aged 5–9.



In [ ]:
population

The same data could be represented in the following decidedly *un*-tidy table.

In [ ]:
population_wide

Instead of having the different age groups be *values* in the age column, the table spreads them across *multiple rows*. This table can probably be made sense of by a human—for example, in 1850, there were 1,483,789 males in the US aged 0–4.

However, it really doesn't lend itself to useful programmatic or visual analyses.

## 3.1: Reshaping Dataframes

Polars provides the `pivot` and `unpivot` functions to help with reshaping data tables. Using the linked documentation as a guide, complete the following steps.

 Return a table that looks identical to `population_wide`.

 *Hint*: You don't have to type out all the column names in an array, and you can check with `.equals` method fo dataframes

In [ ]:
answer_3_1 = ...

# Your code here

answer_3_1

## 3.2: Folding, and merging multiple datasets

Consider the following three SUPER UNTIDY tables from the [Gapminder](https://www.gapminder.org/data/) project.

First, we have data about countries' per-capita CO2 emissions (in tonnes). The table is very wide...



In [ ]:
emissions_untidy

Next, we have data about countries' populations and projected populations from 1800–2100. (Not to be confused with the US population dataset we played with in Part 3.1.)

In [ ]:
pop_untidy

Finally, we have data (and projections) about the countries' GDP per capita (in inflation adjusted 2017 USD) from 1800–2100.

In [ ]:
gdp_untidy

Our goal in this part is to all of this data into one tidy dataset. That is, we want a table that looks something like this.



| Country     | Year | Emissions | GDP | Population |
|-------------|------|-----------|-----|-------|
| Afghanistan | 1800 | 0.001 | 481 | 3.28M |
| Afghanistan | 1801 | 0.001 | 481 | 3.28M |
| Afghanistan | 1803 | 0.001 | 481 | 3.28M |
| Angola      | 1800 | 0.009 | 373 | 1.57M |
| Angola      | 1801 | 0.009 | 374 | 1.57M |
| Angola      | 1802 | 0.009 | 376 | 1.57M |
|	and |	so 	| on |	... |


Where each row is an observation (a specific country's values in a specific year) and each column is an attribute.

In order to do this you will need to first tidy up the individual tables of each data source. Then you will need to combine the individual tables together into one tidy dataframe. Consider the [`join`](https://docs.pola.rs/user-guide/transformations/joins/) operation, and make sure to be careful about the kind of join used, as not all of the tables have data for all of the same years. For example, what if one table had no CO2 data for a given year and country? Should that year be omitted entirely from the merged table? Or should those cells be given empty values?

Similarly, what if the columns were named different things, like year in one table and emissionYear in another table? The various join methods provide options for handling these cases. Please give them a brief look in case you encounter these situations in future assignments.

In [ ]:
answer_3_2 = ...

# Your code here

answer_3_2

## 3.3: Cleaning and Wrangling Data

Finally, you'll notice that the numerical fields don't really have numbers in them. The original data source (Gapminder) has provided the bigger numbers as strings, i.e., instead of the number 3280000, the table contains the value "3.28M".

This makes things easy for a human to read, but really isn't going to work for visualization.




The work of transforming, processing, and cleaning data to make it work for visualization and analysis is generally called *data wrangling* and in most data science and data engineering work takes up a shockingly large percentage of your actual time and effort. Polars provides many powerful and effeicent ways to do much of this work, in particular, ensure that you have fully understood Polars [expressions and contexts](https://docs.pola.rs/user-guide/concepts/expressions-and-contexts/).


In the function below, define a polars **expression** that when run in a `with_columns` context will transform a series of strings such as "3.28M" into a series of numbers of a given datatype (either floats or integers).

In [ ]:
def parse_number_shorthand(series_name, dtype):
  answer_3_3 = ...

  # Your code here

  return answer_3_3

In [ ]:
gapminder_tidy = answer_3_2.with_columns(
    parse_number_shorthand('year',pl.Int32),
    parse_number_shorthand('population',pl.Int32),
    parse_number_shorthand('gdp_per_capita',pl.Int32),
    parse_number_shorthand('emissions_per_capita',pl.Float32),
)
gapminder_tidy

Output should look somthing like this


| country	| year	| population| 	emissions_per_capita	|gdp_per_capita|
|--|--|--|--|--|
|str|	i32	|i32	|f32	|i32|
|--|--|--|--|--|
|"Afghanistan"|	1800	|3280000|	0.001	|481|
|"Angola"|	1800|	1570000|	0.009|	373|
|"Albania"|	1800|	400000|	0.001|	469|
|"Andorra"|1800|	2650|	0.333	|1370|
|"UAE"|	1800|	40200|	0.063|	1140|
|...|...|...|...|...|

## 3.4: Summarizing Data

Now that we have wrangled our data into a useable form we can start to do some analysis and data summarization. Here calculate the total cumulative overall (**not** per capita) CO2 emissions from the USA from 1800-2022.

In [ ]:
answer_3_4 = ...

# Your code here

answer_3_4

# Using Tidy Data for Interactive Visualization

At last now we have built a dataframe that can be used to asnwer questions about our data, and can be integrated with interactive data visualization libraries like Altair. Run the cell below for an example of a reporoduction of the [famous gapminder animation](https://www.youtube.com/watch?v=hVimVzgtD6w), where you can use the slider to progress through the years, hover over circles to see the countries they represent, and zoom in to find interesting patters. I recomend you spend some time going over the source code as well, and play around with the choices I made in order to start learning Altair as it will be the primary python based library we will be using.

In [ ]:
# @title Gapminder Interactive Animation
year_slider = alt.binding_range(min=1950,max=2022, name='Year')
selector = alt.selection_point(name='year_select',fields=['year'], bind=year_slider,value=1950)

scatter = alt.Chart(gapminder_tidy).mark_circle(opacity=0.80).encode(
        x=(alt.X('gdp_per_capita')
              .sort(field= 'population', order= 'descending')
              .scale(type="log",domain=(100, 1000000))
              .axis(grid=False)
              .title('GDP per capita, log scale')
          ),
        y=(alt.Y('emissions_per_capita')
              .sort(field= 'population', order= 'descending')
              .scale(type="log",domain=(0.001, 100))
              .axis(grid=False)
              .title('CO2 emissions (tonnes per capita), log scale')
          ),
        color=(alt.Color('country',legend=None)
              .sort(field= 'country', order= 'descending')
          ),
        size=(alt.Size('population',legend=None)
              .scale(range=(10, 10000))
          ),
        tooltip=['country','population','emissions_per_capita']
).add_params(
    selector
).transform_filter(
    selector
)

background_year = alt.Chart(gapminder_tidy).mark_text(
    baseline='middle',
    fontSize=75,
    opacity=0.1
).encode(
    text='year:O'
).transform_filter(
    selector
).transform_aggregate(
    year='max(year)'
)


chart = alt.layer(
    scatter, background_year
).properties(
    width=500,
    height=500,
    padding=10  # Padding ensures labels fit
).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).interactive()

chart

